In [1]:
import sqlite3
import pandas as pd
import numpy as np
import time
from utils import *
from ReactNode import ReactNode
from ChemNode import ChemNode
from MCTS import MCTS
from tree_utils import *

In [2]:
conn = sqlite3.connect('buyable.db')
buyable = conn.cursor()
retrobiocat = pd.read_pickle("data/retrobiocat_db_added_template.pkl")
analyzer = Retrosim()

In [3]:
smile = "C#C[C@]1([C@H](C[C@@H](O1)N2C=NC3=C(N=C(N=C32)F)N)O)CO"
root = ChemNode(smile, 0, None, True, buyable, retrobiocat, analyzer, None)

In [4]:
mcts = MCTS(root, 3000)

In [5]:
start = time.time()
mcts.MCTS()
print("Time taken (s): ", time.time()-start)


Time taken (s):  1035.9324295520782


In [6]:
for react in root.reactions:
    print(f'Reaction name: {react.reaction_name}, Score: {react.get_mcts_value()}')
    for precursor in react.precursors:
        print(f'Chem {precursor.smiles}, Score: {precursor.score}, visits: {precursor.visits}, solution: {precursor.solution}')
    print(" ")

Reaction name: Phosphate hydrolysis, Score: 1.0616769890375268
Chem C#C[C@]1(CO)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1OP(=O)(O)O, Score: 1.8525, visits: 17, solution: False
 
Reaction name: Phosphate hydrolysis, Score: 1.0775667471062595
Chem C#C[C@]1(COP(=O)(O)O)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O, Score: 0.95, visits: 15, solution: False
 
Reaction name: Ester hydrolysis (OH), Score: 1.0775667471062595
Chem C#C[C@]1(COC(=O)CC)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O, Score: 0.95, visits: 15, solution: False
 
Reaction name: Ester hydrolysis (OH), Score: 1.0142334137729263
Chem C#C[C@]1(CO)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1OC(=O)CC, Score: 0.0, visits: 15, solution: False
 
Reaction name: Ketone reduction, Score: 1.0498313219683797
Chem C#C[C@]1(CO)O[C@@H](n2cnc3c(N)nc(F)nc32)CC1=O, Score: 0.0, visits: 14, solution: False
 
Reaction name: Ene hydrolysis, Score: 1.0498313219683797
Chem C#C[C@@]1(CO)C=C[C@H](n2cnc3c(N)nc(F)nc32)O1, Score: 0.0, visits: 14, solution: False
 
Reaction name:

In [7]:
root.solution

False

In [21]:
# def prune_tree(node: ChemNode):
#     """
#     Prune the tree by removing all non-solution pathways.

#     :param node: The root ChemNode to prune.
#     :return: None (modifies the tree in-place).
#     """
#     # Iterate over a copy of the reactions to safely modify the original list
#     for reaction in node.reactions[:]:
#         # Check if all reagents in the reaction are solutions
#         if all(reagent.solution for reagent in reaction.precursors):
#             # Recursively prune each reagent
#             for reagent in reaction.precursors:
#                 prune_tree(reagent)
#         else:
#             # Remove non-solution reagents from the reaction
#             for reagent in reaction.precursors[:]:
#                 if not reagent.solution:
#                     reaction.precursors.remove(reagent)
            
#             # If the reaction has no reagents left, remove the reaction
#             if not reaction.precursors:
#                 node.reactions.remove(reaction)

#     # If the node itself is not a solution and has no reactions, prune it at the parent level
#     if not node.solution and not node.reactions:
#         return False

# # Plan: Go down tree, copying along a given path to add to a single path list

# def generate_subtrees(node: ChemNode):
#     # Base case: if the node is a leaf
#     if node.solution and not node.reactions: # Leaf node
#         node.solution = False
#         return node.copy()
    
#     copy = node.copy()
#     # If the node is not a leaf, generate subtrees for all reactions
       
#     reaction = node.reactions[0]
#     react = reaction.copy()
#     for reagent in reaction.precursors:
#         if reagent.solution:
#             react.add_reagent(generate_subtrees(reagent))

#     if all(not reagent.solution for reagent in node.reactions[0].precursors):
#         node.reactions.pop(0)

#     if not node.reactions:
#         node.solution = False

    

#     copy.reactions.append(react)
#     return copy

# def generate_paths(root: ChemNode):
#     paths = []
#     while root.solution:
#         paths.append(generate_subtrees(root))
#     return paths

# def path_explorer(root: ChemNode):
#     """
#     Prints the path given a subtree
#     """

#     stack = []
#     stack.append(root)
#     count = 0
#     while stack:
#         print(f"BRANCH {count}:\n-----------------")
#         count += 1
#         root = stack.pop()
#         print(f'Chem: {root.smiles}')
#         while root.reactions:
#             print(f'Reaction: {root.reactions[0].reaction_name}')
#             if len(root.reactions[0].precursors) > 1:
#                 print(f"SPLIT: Chem1: {root.reactions[0].precursors[0].smiles}, Chem2: {root.reactions[0].precursors[1].smiles}\n")
#                 stack.append(root.reactions[0].precursors[1])
#             else:
#                 print(f'Chem: {root.reactions[0].precursors[0].smiles}')
#             root = root.reactions[0].precursors[0]
#         print("BUYABLE\n")
        

In [22]:
# Prune the tree starting from the root
prune_tree(root)
paths = generate_paths(root)
print(len(paths))

1


In [23]:
path_explorer(paths[0])

BRANCH 0:
-----------------
Chem: C#C[C@]1([C@H](C[C@@H](O1)N2C=NC3=C(N=C(N=C32)F)N)O)CO
Reaction: PNP
SPLIT: Chem1: C#C[C@]1(CO)O[C@@H](OP(=O)([O-])O)C[C@@H]1O, Chem2: Nc1nc(F)nc2[nH]cnc12

Reaction: Phosphate Transfer
Chem: C#C[C@]1(COP(=O)([O-])O)O[C@@H](O)C[C@@H]1O
Reaction: Hemiacetal closure
Chem: C#C[C@@](O)(COP(=O)([O-])O)[C@@H](O)CC=O
Reaction: Aldol reaction (DERA)
SPLIT: Chem1: C#C[C@](O)(C=O)COP(=O)([O-])O, Chem2: CC=O

BUYABLE

BRANCH 1:
-----------------
Chem: CC=O
BUYABLE

BRANCH 2:
-----------------
Chem: Nc1nc(F)nc2[nH]cnc12
BUYABLE

